In [9]:
import torch
import torch.nn.functional as F
torch.__version__

'1.6.0'

# 2.4 卷积神经网络简介

卷积神经网络由**一个或多个卷积层和顶端的全连通层**（也可用1x1的卷积层作为最终输出）组成**前馈神经网络**。

1. 一般认为，卷积神经网络由Yann LeCun大神在1989年提出的**LeNet**中首先被使用。但是由于当时计算能力不够，并没有得到广泛应用。


1. 1998年Yann LeCun及其合作者构建了更加完备的卷积神经网络LeNet-5，并在手写数字识别中取得成功。LeNet-5的成功使卷积神经网络得到关注。LeNet-5沿用LeCun (1989) 的学习策略，并在原有设计中加入池化层对输入特征筛选。LeNet-5基本定义了现代卷积神经网络的基本结构，交替出现的卷积层-池化层，有效提取输入图像的平移不变特征，使得特征提取前进一大步。一般认为Yann LeCun是卷积神经网络的创始人。


1. 2006年后，随着深度学习理论完善，尤其是计算能力提升和**参数微调fine-tuning** 等技术的出现，卷积神经网络开始快速发展。结构不断加深，各类学习和优化理论引入。2012年的**AlexNet**、2014年的**VGGNet、GoogLeNet**和2015年的**ResNet**,使得卷积神经网络几乎成为深度学习图像处理的标配。

## 2.4.1 为什么要用卷积神经网络

计算机视觉，每个图像由一个个像素点构成。每个像素点有三个通道，分别代表RGB三种颜色(不计算透明度)。

手写识别数据集MNIST，每个图像是一个长宽均为28，channel为1的单色图像。

如果用**全连接网络结构**，即网络神经与相邻层的每个神经元均连接，意味网络有**28*28=784个神经元（RGB3还要*3）**，
**hidden层**如果用**15个神经元**，参数个数(w和b)就有：**28 * 28 * 15 * 10 + 15 + 10=117625个** 。

这个数量级到现在为止也是一个很恐怖的数量级，一次反向传播计算量巨大。

这还只是一个单色的28像素大小的图片，如果用更大的像素计算量可想而知。

## 2.4.2结构组成

* 传统网络需要大量参数，但参数是否重复。例如识别一个人，只要看到眼睛鼻子嘴和脸，基本就知道这个人是谁。只用这些局部特征就能做判断，不需要所有特征。

* 有效提取输入图像的**平移不变特征**，眼睛在左边还是右边，都是眼睛。

* 通过**卷积操作**提取图像局部特征，每一层计算出一些局部特征。这些局部特征汇总到下一层，一层一层传递，特征由小变大。最后通过这些局部特征处理图片，大大提高效率和准确度。

### 卷积层
#### 卷积计算
卷积计算，[知乎](https://www.zhihu.com/question/39022858)上的一张图片

<img src="9.gif" width=400>

* 定义**权重矩阵W**（**卷积核kernel**,也称过滤器filter）。

* 权重矩阵大小一般为`3 * 3` 或`5 * 5`。LeNet用到比较大的`7 * 7`，现在已经很少见。根据经验验证，**3和5是最佳大小**。

* 用权重矩阵在输入矩阵上滑动。每滑动一步，将覆盖值与矩阵对应值相乘，将结果求和作为输出矩阵的一项。依次类推直到完成全部计算。

* 上图输入`5 * 5`矩阵，通过`3 * 3`卷积核，计算结果是一个`3 * 3`的新矩阵。新矩阵的大小如何计算？

#### 卷积核大小 f
核大小用f表示

#### 边界填充 (p)adding
上图经过计算，矩阵大小改变。如果要使矩阵大小不变，可以先对矩阵**填充**。
将矩阵周围包围一层，矩阵变成`7*7`,上下左右各加1，相当于 `5+1+1=7`。计算结果还是`5 * 5`矩阵，保证大小不变，这里p=1。

#### 步长 (s)tride
每次滑动只是滑动一个距离，如果每次滑动两个距离，就要使用步长这个参数。

#### 计算公式
n为输入矩阵的大小，$ \frac{n-f+2p}{s} +1 $ 向下取整，这个公式非常重要一定要记住。

#### 卷积层
每个卷积层设置**多个核**，每个核代表**不同特征**。这些特征传递到下一层输出，训练过程就是训练这些不同核。

### 激活函数
卷积操作是线性的，所以也要激活。一般情况下会使用**relu**。


### 池化层（pooling）
**池化层**是CNN的重要组成部分，通过减少卷积层间的连接，**降低运算复杂度**。
池化操作简单，相当于合并。输入过滤器的大小，与卷积操作一样一步步滑动，过滤器覆盖区域合并，只保留一个值。
合并方式也有很多种，如常用的两种：**取最大值maxpooling，取平均值avgpooling**

池化层的输出大小公式也与卷积层一样，由于没有填充，所以p=0，可以简化为
$ \frac{n-f}{s} +1 $

### dropout层
dropout是2014年 Hinton 提出，防止过拟合采用的trick，**增强模型的泛化能力**。

**Dropout（随机失活）**：

深度学习网络训练过程中，按照一定概率将一部分神经单元**暂时从网络中丢弃**，
相当于从原始网络中找到更瘦网络，**随机将一部分网络的传播掐断**。
听起来好像不靠谱，但是实际测试效果非常好。原文[Dropout: A Simple Way to Prevent Neural Networks from Overfitting](http://jmlr.org/papers/v15/srivastava14a.html)

### 全连接层
**全连接层**一般作为**输出层**，卷积提取图像特征，最后的全连接层通过这些特征计算，输出所要结果。
无论是分类，还是回归。

**特征**用矩阵表示，传入全连接层前，还需要对特征压扁，将特征变成一维向量。

如果分类，用**sofmax作为输出**，如果回归，直接使用**linear**。

以上就是卷积神经网络主要的组成部分，下面介绍一些经典网络模型

## 2.4.3 经典模型

### LeNet-5

1998， Yann LeCun 的 LeNet5 [官网](http://yann.lecun.com/exdb/lenet/index.html)

卷积神经网路的开山之作，麻雀虽小但五脏俱全，**卷积层、pooling层、全连接层**，是现代CNN网络的基本组件
   - 卷积提取空间特征；
   - 空间平均得到子样本；
   - tanh 或 sigmoid 得到非线性；
   - multi-layer neural network（MLP）作为最终分类器；
   - 层层之间用稀疏连接矩阵，以避免大的计算成本。
![](lenet5.jpg)

**输入：**

图像Size为32*32，比mnist数据库最大字母(28*28)还大。目的是希望潜在的**明显特征**，如笔画断续、角点出现在最高层特征监测子感受野中心。

**输出：**

10个类别，为0-9数字的概率

1. C1层是卷积层，有6个卷积核（提取6种局部特征），核大小为5 * 5
2. S2层是pooling层，下采样（区域:2 * 2 ）降低网络训练参数及模型过拟合程度
3. C3层是第二个卷积层，使用16个卷积核提取特征，核大小:5 * 5 
4. S4层也是一个pooling层，区域:2*2
5. C5层是最后一个卷积层，卷积核种类:120，卷积核大小:5 * 5  
6. 最后用全连接层，将C5的120个特征进行分类，最后输出0-9的概率

代码来自[官方教程](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)

In [10]:
import torch.nn as nn


class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution

        # 卷积层1
        self.conv1 = nn.Conv2d(1, 6, 5)

        # 卷积层2
        self.conv2 = nn.Conv2d(6, 16, 5)

        # 论文写的是conv,官方教程用线性层
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):

        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))

        # If the size is a square, you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):

        # all dimensions except the batch dimension
        size = x.size()[1:]

        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = LeNet5()
print(net)

LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
x = torch.rand(1, 1, 32, 32)
nn1 = LeNet5()
print(nn1)
out = nn1(x)
print(out)

LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
tensor([[ 0.0377,  0.0211,  0.0775,  0.0949,  0.0233, -0.0655,  0.0476, -0.1043,
         -0.0633, -0.0328]], grad_fn=<AddmmBackward>)


### AlexNet

2012，Alex Krizhevsky 算作LeNet的更深和更广的版本，用来学习更复杂的对象 [论文](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)

   - rectified linear units（ReLU）得到非线性；
   - dropout 技巧在训练期间选择性忽略单个神经元，减缓模型的过拟合；
   - 重叠最大池，避免平均池的平均效果；
   - GPU NVIDIA GTX 580 减少训练时间，比用CPU处理快 10 倍，可以被用于更大的数据集和图像上。
   
![](alexnet.png)

虽然 AlexNet只有8层，但有60M以上的参数总量，

Alexnet有一个特殊的计算层**LRN层**，对当前层的输出结果做平滑处理。Alexnet的每一阶段（含一次卷积计算的算作一层）可以分为8层：

1. con - relu - pooling - LRN ：
input层是227\*227，而不是paper里的224。227可以整除conv1计算，224不整除。
如果一定要用224,可以通过自动补边。input补边没有意义，补的也是0，这就是公式的重要性。

2. conv - relu - pool - LRN ：
group=2，这个属性强行把前面结果的**feature map分开**，卷积部分分成两部分做

3. conv - relu

4. conv - relu

5. conv - relu - pool

6. fc - relu - dropout ：
dropout层，Alexnet在训练中以1/2概率使隐藏层的某些neuron输出为0，丢掉一半节点的输出。
BP时也不更新这些节点，防止过拟合。

7. fc - relu - dropout 

8. fc - softmax 

Pytorch的vision包中包含Alexnet官方实现，直接使用官方版本看下网络

In [12]:
import torchvision

# 不下载预训练权重
model = torchvision.models.alexnet(pretrained=False)
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### VGG

2015，牛津的 VGG。[论文](https://arxiv.org/pdf/1409.1556.pdf)

   - 每个卷积层中用更小的 3×3 filters，组合成卷积序列
   - 多个3×3卷积序列模拟更大的接收场效果
   - 每次的图像像素缩小一倍，卷积核数量增加一倍
 
VGG有很多个版本，算是比较稳定和经典的model。
特点是连续conv多，计算量巨大，这里以VGG16为例.
[图片来源](https://www.cs.toronto.edu/~frossard/post/vgg16/)
![](vgg16.png) 

VGG清一色用小卷积核，小卷积核比用大卷积核的优势：

* input=8 -> 3层conv3x3后，output=2，等同于1层conv7x7的结果；
* input=8 -> 2层conv3x3后，output=2，等同于2层conv5x5的结果

卷积层参数减少。相比5x5、7x7和11x11的大卷积核，3x3明显减少参数量

通过卷积和池化层后，图像分辨率降低为原来的一半。但是图像特征增加一倍，这是个十分规整的操作:

分辨率由输入的224->112->56->28->14->7，
特征从原始RGB3个通道-> 64 ->128 -> 256 -> 512

这为后面网络提供一个标准，依旧使用Pytorch官方实现版本查看

In [13]:
import torchvision

#不下载预训练权重
model = torchvision.models.vgg16(pretrained=False)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### GoogLeNet (Inception)
2014，Google Christian Szegedy [论文](https://arxiv.org/abs/1512.00567)

- 使用1×1卷积块（NiN）减少特征数量，通常被称为“瓶颈”，减少深层神经网络的计算负担。
- 每个池化层前增加 feature maps，增加每一层的宽度，增多特征的组合性

googlenet最大特点是包含若干个**inception模块**，有时也称作 inception net。
googlenet虽然层数比VGG多很多，但由于inception设计，计算速度快很多。
<img src="googlenet.png" width=400>

Inception架构主要思想： 找出如何让已有稠密组件接近与覆盖卷积视觉网络中的最佳局部稀疏结构。
现在需要找出最优的局部构造，并且重复几次。之前的一篇文献提出一个层与层的结构，在最后一层进行相关性统计，将高相关性的聚集到一起。
这些聚类构成下一层的单元，且与上一层单元连接。假设前面层的每个单元对应于输入图像的某些区域，这些单元被分为滤波器组。
在接近输入层的低层中，相关单元集中在某些局部区域，最终得到在单个区域中的大量聚类，在最后一层通过1x1的卷积覆盖。

上面的话听起来很生硬，其实解释起来很简单：

* 每一模块都用若干个不同的特征提取方式，例如 3x3卷积，5x5卷积，1x1卷积，pooling等，都计算一下

* 最后把这些结果通过Filter Concat进行连接，找到这里面作用最大的

* 网络里包含许多这样的模块，不用人为判断哪个特征提取方式好，网络自己解决（有点像AUTO ML）。

* Pytorch实现了InceptionA-E，还有InceptionAUX 模块。

In [14]:
# inception_v3需要scipy，pip install scipy

import torchvision

# 不下载预训练权重
model = torchvision.models.inception_v3(pretrained=False)
print(model)

/Users/jinminyu/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/inception.py:75: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of inception_v3 will be changed in future releases of '


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

### ResNet

2015 Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun [论文](https://arxiv.org/abs/1512.03385)
Kaiming He 何凯明（音译）大神，很多论文都有他参与(mask rcnn, focal loss)。

Jian Sun孙剑老师，现在旷世科技的首席科学家。

googlenet已经很深，ResNet可以做到更深，通过残差计算可以训练超过1000层网络，俗称**跳连接**。

#### 退化问题
网络层数增加，但训练集的准确率却饱和甚至下降。
不能解释为overfitting，因为overfiting应该表现为训练集表现更好。
这个是**网络退化问题**，退化说明深度网络不能很简单地被很好优化

#### 残差网络的解决办法
深层网络的后面那些层是恒等映射，模型退化为一个浅层网络。
现在要解决的是学习恒等映射函数。让一些层拟合一个潜在的恒等映射函数H(x) = x比较困难。
如果把网络设计为 H(x) = F(x) + x，可以转换为学习一个残差函数F(x) = H(x) - x。 
只要F(x)=0，就构成一个恒等映射H(x) = x，拟合残差更加容易。

以上又很不好理解，继续解释下，先看图：![](resnet.png)

激活函数前将上一层（或几层）的输出与本层计算的输出相加，将求和的结果输入到激活函数中做为本层的输出，
引入残差后的映射对输出的变化更敏感，其实就是看本层相对前几层是否有大的变化，相当于是一个差分放大器的作用。
图中的曲线就是残差中的shoutcut，将前一层的结果直接连接到了本层，也就是俗称的跳连接。

经典的resnet18看一下网络结构 [图片来源](https://www.researchgate.net/figure/Proposed-Modified-ResNet-18-architecture-for-Bangla-HCR-In-the-diagram-conv-stands-for_fig1_323063171)
![](resnet18.jpg)

In [15]:
import torchvision

#不下载预训练权重
model = torchvision.models.resnet18(pretrained=False)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

那么该如何选择网络呢？
[来源](https://www.researchgate.net/figure/Comparison-of-popular-CNN-architectures-The-vertical-axis-shows-top-1-accuracy-on_fig2_320084139)
![](cnn.png)
以上表格可以清楚看到准确率和计算量间的对比。

建议是小型图片分类任务，resnet18基本可以。如果对准确度要求比较高，再选其他更好的网络架构。

**另外有句俗话叫：穷人只能alexnet，富人才用Res**